In [1]:
# import relevant modules
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import os
import io
import pydotplus
import imageio
import itertools
import time

from scipy import misc
from scipy import stats
from IPython.display import Image
from sklearn import tree  
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.externals.six import StringIO  
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Settings
pd.set_option('display.max_columns', None)
np.set_printoptions(precision=3)
sns.set(style="darkgrid")
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12



C:\Program Files\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [2]:
#Dataset loading
#FullDataframe = pd.read_csv("D:/DataSets/CIC-IDS-2017/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv") 

"""
from glob import glob
dfs = []
for file in sorted(glob('"D:/DataSets/CIC-IDS-2017/MachineLearningCVE/*.csv')):
    print(file)
    dfs.append(pd.read_csv(file))
FullDataframe = pd.concat(dfs)
"""

df1=pd.read_csv("D:/DataSets/CIC-IDS-2017/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df2=pd.read_csv("D:/DataSets/CIC-IDS-2017/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df3=pd.read_csv("D:/DataSets/CIC-IDS-2017/MachineLearningCVE/Friday-WorkingHours-Morning-Botnet.pcap_ISCX.csv")
df4=pd.read_csv("D:/DataSets/CIC-IDS-2017/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv")
df5=pd.read_csv("D:/DataSets/CIC-IDS-2017/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
df6=pd.read_csv("D:/DataSets/CIC-IDS-2017/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
df7=pd.read_csv("D:/DataSets/CIC-IDS-2017/MachineLearningCVE/Tuesday-WorkingHours-Bruteforce.pcap_ISCX.csv")
df8=pd.read_csv("D:/DataSets/CIC-IDS-2017/MachineLearningCVE/Wednesday-workingHours-DoS.pcap_ISCX.csv")

FullDataframe = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8])
print("Original version has {} rows & {} columns".format(FullDataframe.shape[0],FullDataframe.shape[1]))
del df1,df2,df3,df4,df5,df6,df7,df8
FullDataframe = FullDataframe.drop_duplicates()
print("No-duplicates version has {} rows & {} columns".format(FullDataframe.shape[0],FullDataframe.shape[1]))

Original version has 2830743 rows & 79 columns
No-duplicates version has 2522362 rows & 79 columns


In [3]:
train, test = train_test_split(FullDataframe, test_size = 0.20, random_state=30)

train['Label'].value_counts()

BENIGN                        1677271
DoS Hulk                       138132
DDoS                           102537
PortScan                        72591
DoS GoldenEye                    8266
FTP-Patator                      4770
DoS slowloris                    4319
DoS Slowhttptest                 4144
SSH-Patator                      2562
Bot                              1552
Web Attack � Brute Force         1177
Web Attack � XSS                  513
Infiltration                       30
Web Attack � Sql Injection         18
Heartbleed                          7
Name: Label, dtype: int64

In [4]:
train_y = train['Label']
train_x = train.drop(['Label'], axis=1)
#train_y

In [5]:
train_x2 = train_x
train_x = train_x.values.astype(np.long)
np.nan_to_num(train_x)

#train_x2

array([[   64167, 11406268,        1, ...,        0, 11400000, 11400000],
       [     443,    96291,        7, ...,        0,        0,        0],
       [     443,       78,        2, ...,        0,        0,        0],
       ...,
       [      80,     4318,        2, ...,        0,        0,        0],
       [      53,      299,        2, ...,        0,        0,        0],
       [      53,    30595,        2, ...,        0,        0,        0]])

In [6]:
from sklearn.svm import SVC 
from sklearn.naive_bayes import BernoulliNB 
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
print(time.ctime())
# Train KNeighborsClassifier Model
KNN_Classifier = KNeighborsClassifier(n_jobs=-1)
KNN_Classifier.fit(train_x, train_y); 
print(time.ctime())
# Train LogisticRegression Model
LGR_Classifier = LogisticRegression(n_jobs=-1, random_state=0)
LGR_Classifier.fit(train_x, train_y);
print(time.ctime())
# Train Gaussian Naive Baye Model
BNB_Classifier = BernoulliNB()
BNB_Classifier.fit(train_x, train_y)
print(time.ctime())
# Train Decision Tree Model
DTC_Classifier = tree.DecisionTreeClassifier(criterion='entropy', random_state=0)
DTC_Classifier.fit(train_x, train_y)
print(time.ctime())

Sat Jun 27 15:32:14 2020
Sat Jun 27 15:41:07 2020
Sat Jun 27 15:47:52 2020
Sat Jun 27 15:48:12 2020
Sat Jun 27 15:50:10 2020


In [7]:
from sklearn import metrics

models = []
models.append(('Naive_Baye', BNB_Classifier))
models.append(('Decision_Tree', DTC_Classifier))
models.append(('KNeighbors', KNN_Classifier))
models.append(('Logistic_Regression', LGR_Classifier))
#models.append(('Random_Forest', RFC_Classifier))

for i, v in models:
    print(time.ctime())
    scores = cross_val_score(v, train_x, train_y, cv=100)
    accuracy = metrics.accuracy_score(train_y, v.predict(train_x))
    confusion_matrix = metrics.confusion_matrix(train_y, v.predict(train_x))
    classification = metrics.classification_report(train_y, v.predict(train_x))
    print(time.ctime())
    print()
    print('============================== {} Model Evaluation =============================='.format(i))
    print()
    print ("Cross Validation Mean Score:" "\n", scores.mean())
    print()
    print ("Model Accuracy:" "\n", accuracy)
    print()
    print("Confusion matrix:" "\n", confusion_matrix)
    print()
    print("Classification report:" "\n", classification) 
    print()


Sat Jun 27 15:50:10 2020
Sat Jun 27 17:01:25 2020

============================== Naive_Baye Model Evaluation ==============================

Cross Validation Mean Score:
 0.6008759687328297

Model Accuracy:
 0.6009195748626411

Confusion matrix:
 [[1002825    6853   44849    2488  216572   54611    3074   17382     267
    19611    1607  152205  123966    9268   21693]
 [    567     391       0       0       1       0       0       0       0
        0       0     591       2       0       0]
 [    124      48   15912       0   18599   21306       0      13       4
        0       0   46531       0       0       0]
 [    284       0     569       0    4614    2059       0     141       0
        0       0     599       0       0       0]
 [   6547       1    1118       0  112737    1559       5       9       1
        0       3   11206     403       8    4535]
 [    347     212     148     797     114    2212     296       1       0
        0       0       2       3       0      12]
 [

Sun Jun 28 20:09:26 2020

============================== Logistic_Regression Model Evaluation ==============================

Cross Validation Mean Score:
 0.823759875627162

Model Accuracy:
 0.8191520940943728

Confusion matrix:
 [[1465907       0   44360     598   21106  143305    1937       0       0
        0       0      58       0       0       0]
 [   1518       0      34       0       0       0       0       0       0
        0       0       0       0       0       0]
 [  19816       0   78783       0    3938       0       0       0       0
        0       0       0       0       0       0]
 [   8205       0      16       2       9      34       0       0       0
        0       0       0       0       0       0]
 [  12844       0   13588      30  107390    4280       0       0       0
        0       0       0       0       0       0]
 [   3088       0     201       0       0     855       0       0       0
        0       0       0       0       0       0]
 [   4161       0  

In [8]:
test_y = test['Label']
test_x = test.drop(['Label'], axis=1)
test_y
test_x

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
283776,53,200,2,2,68,158,34,34,34.000000,0.000000,79,79,79.000000,0.000000,1.130000e+06,20000.000000,6.666667e+01,3.233162e+01,104,48,48,48.00,0.000000e+00,48,48,48,4.800000e+01,0.000000e+00,48,48,0,0,0,0,64,64,10000.000000,10000.000000,34,79,52.000000,24.647515,6.075000e+02,0,0,0,0,0,0,0,0,1,65.000000,34.000000,79.000000,64,0,0,0,0,0,0,2,68,2,158,-1,-1,1,32,0.0,0.0,0,0,0.0,0.0,0,0
498876,53,94910,1,1,49,112,49,49,49.000000,0.000000,112,112,112.000000,0.000000,1.696344e+03,21.072595,9.491000e+04,0.000000e+00,94910,94910,0,0.00,0.000000e+00,0,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,20,40,10.536298,10.536298,49,112,70.000000,36.373067,1.323000e+03,0,0,0,0,0,0,0,0,1,105.000000,49.000000,112.000000,20,0,0,0,0,0,0,1,49,1,112,-1,-1,0,20,0.0,0.0,0,0,0.0,0.0,0,0
163061,51433,2,2,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,1000000.000000,2.000000e+00,0.000000e+00,2,2,2,2.00,0.000000e+00,2,2,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,64,0,1000000.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,0,0,0,0,1,0,0,0,0,0.000000,0.000000,0.000000,64,0,0,0,0,0,0,2,0,0,0,4988,-1,0,32,0.0,0.0,0,0,0.0,0.0,0,0
263134,80,86069741,7,7,425,11595,407,0,60.714286,152.727114,5792,0,1656.428571,2119.480114,1.396542e+02,0.162659,6.620749e+06,2.360000e+07,85100000,3,86100000,14300000.00,3.460000e+07,85100000,758,86100000,1.430000e+07,3.510000e+07,86100000,50,0,0,0,0,204,232,0.081329,0.081329,0,5792,801.733333,1618.721121,2.620258e+06,1,0,0,0,0,0,0,0,1,859.000000,60.714286,1656.428571,204,0,0,0,0,0,0,7,425,7,11595,0,235,3,20,11043.0,0.0,11043,11043,85100000.0,0.0,85100000,85100000
84077,53,66310,1,1,50,137,50,50,50.000000,0.000000,137,137,137.000000,0.000000,2.820087e+03,30.161363,6.631000e+04,0.000000e+00,66310,66310,0,0.00,0.000000e+00,0,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,32,40,15.080682,15.080682,50,137,79.000000,50.229473,2.523000e+03,0,0,0,0,0,0,0,0,1,118.500000,50.000000,137.000000,32,0,0,0,0,0,0,1,50,1,137,-1,-1,0,32,0.0,0.0,0,0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63398,21,8469324,9,15,107,188,23,0,11.888889,9.584420,34,0,12.533333,14.549750,3.483159e+01,2.833756,3.682315e+05,9.566314e+05,3018141,3,5858078,732259.75,1.355889e+06,3062999,3,8469245,6.049461e+05,1.180892e+06,3018141,3,0,0,0,0,296,488,1.062659,1.771098,0,34,11.800000,12.658989,1.602500e+02,0,0,0,1,0,0,0,0,1,12.291667,11.888889,12.533333,296,0,0,0,0,0,0,9,107,15,188,29200,227,6,32,0.0,0.0,0,0,0.0,0

In [9]:
test_x = test_x.values.astype(np.long)
np.nan_to_num(test_x)

array([[      53,      200,        2, ...,        0,        0,        0],
       [      53,    94910,        1, ...,        0,        0,        0],
       [   51433,        2,        2, ...,        0,        0,        0],
       ...,
       [      80, 99731093,        6, ...,        0, 99600000, 99600000],
       [      53,    55617,        2, ...,        0,        0,        0],
       [      80, 80211780,        8, ..., 46900000, 72400000,  6030874]])

In [10]:
for i, v in models:
    accuracy = metrics.accuracy_score(test_y, v.predict(test_x))
    confusion_matrix = metrics.confusion_matrix(test_y, v.predict(test_x))
    classification = metrics.classification_report(test_y, v.predict(test_x))
    print()
    print('============================== {} Model Test Results =============================='.format(i))
    print()
    print ("Model Accuracy:" "\n", accuracy)
    print()
    print("Confusion matrix:" "\n", confusion_matrix)
    print()
    print("Classification report:" "\n", classification) 
    print()      


============================== Naive_Baye Model Test Results ==============================

Model Accuracy:
 0.6011957032388254

Confusion matrix:
 [[250557   1712  11411    630  54305  13628    794   4277     50   4828
     374  37823  31164   2227   5433]
 [   156     99      0      0      1      0      0      0      0      0
       0    145      0      0      0]
 [    37      9   3959      1   4583   5311      0      3      1      0
       0  11575      0      0      0]
 [    66      0    132      0   1136    513      0     26      0      0
       0    147      0      0      0]
 [  1605      0    296      0  28357    403      3      2      1      0
       0   2781    123      4   1142]
 [    96     62     34    187     27    587     79      1      0      0
       0      2      0      0      9]
 [    36     11      3     41    327      6    291     34      0      0
       0      1      0      0    316]
 [    10      0      0      0      0      3      0    769      0      0
       0


============================== Logistic_Regression Model Test Results ==============================

Model Accuracy:
 0.8183530139373167

Confusion matrix:
 [[366105      0  11159    165   5307  35984    473      0      0      0
       0     20      0      0      0]
 [   387      0     14      0      0      0      0      0      0      0
       0      0      0      0      0]
 [  4972      0  19497      0   1010      0      0      0      0      0
       0      0      0      0      0]
 [  2005      0      6      0      5      4      0      0      0      0
       0      0      0      0      0]
 [  3232      0   3345      9  27027   1104      0      0      0      0
       0      0      0      0      0]
 [   822      0     60      0      0    202      0      0      0      0
       0      0      0      0      0]
 [  1028      0      0      0      0     32      6      0      0      0
       0      0      0      0      0]
 [  1163      0      0      0      0      0      0      0      0      0

In [11]:
pred_knn = KNN_Classifier.predict(test_x)
pred_NB = BNB_Classifier.predict(test_x)
pred_log = LGR_Classifier.predict(test_x)
pred_dt = DTC_Classifier.predict(test_x)
#pred_rf = RFC_Classifier.predict(test_x)

In [12]:
pred_NB

array(['BENIGN', 'BENIGN', 'DoS Slowhttptest', ..., 'DoS Hulk', 'BENIGN',
       'DoS Hulk'], dtype='<U26')

In [13]:
pred_log

array(['BENIGN', 'BENIGN', 'DoS Slowhttptest', ..., 'DoS Hulk', 'BENIGN',
       'DoS Hulk'], dtype=object)

In [14]:
pred_dt

array(['BENIGN', 'BENIGN', 'BENIGN', ..., 'DoS Hulk', 'BENIGN', 'DDoS'],
      dtype=object)

In [15]:
pred_knn

array(['BENIGN', 'BENIGN', 'BENIGN', ..., 'DoS Hulk', 'BENIGN', 'DDoS'],
      dtype=object)

In [16]:
pred_rf

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\ruifr\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-2e0662501732>", line 1, in <module>
    pred_rf
NameError: name 'pred_rf' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\ruifr\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\ruifr\AppData\Roaming\Python\Python37\site-packages\IPython\core\ultratb.py", line 1148, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\Users\ru

NameError: name 'pred_rf' is not defined